# R-L delay tests

[L. Blackburn, Sep 2018]

The purpose of this test is to check stability of R-L delay alignment in one antenna using a reference signal (either polarization) from another antenna. Outliers in R-L delay indicate a mis-fringe in the parallel and/or cross hand polarization product of the baselines being tested. The test is sensitive to delayed leakage in either antenna which can cause a spurious cross-hand fringe. It can also show misidentified singleband delay, e.g. from being outside the search window.

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
from eat.hops import util as hu
from eat.plots import util as pu
from collections import OrderedDict
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.ticker import MultipleLocator
import seaborn as sns
sns.reset_orig()
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']
import sys
nb_stdout = sys.stdout # grab for later
def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()
def tightx(): plt.autoscale(enable=True, axis='x', tight=True)
def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')
def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])
# pd.options.display.float_format = '{:,.6f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))
# Run this in Python once, it should take effect permanently
from notebook.services.config import ConfigManager
c = ConfigManager()
_ = c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})
import string

In [ ]:
os.getcwd()

In [ ]:
# define and load data
alistf = 'alist.v6'
stage = '3.+adhoc'
# datadir = '/data/2017-april/ce/er3/hops-lo/%s/data/' % stage
datadir = os.environ['DATADIR'] + '/'
a = util.noauto(hops.read_alist(datadir + alistf))
util.fix(a)
util.unwrap_mbd(a)
util.add_days(a)
util.add_delayerr(a)
util.add_path(a)
util.add_scanno(a)

In [ ]:
# data filters
a = a[(a.snr > 7.) & ~a.baseline.isin({'RS', 'SR'})].copy()

In [ ]:
# expt_no boundary ticker marks vs scan
elines = a.scan_no.sort_values().values[np.nonzero(np.diff(a.expt_no.sort_values()) > 0)[0]] + 0.5

In [ ]:
# show all output in cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
for site in set(''.join(set(a.baseline))):
    try:
        (p, stats) = hu.rl_segmented(a, site, restarts=hu.restarts_2017)
        hu.rlplot(p, corrected=True)
        multline(elines)
        outliers = ((np.abs(p.LR_offset) > 0.000200) & (np.abs(p.LR_std) > 5)
                    & ~(p.baseline.str.contains('L')
                    & (np.abs(np.abs(p.LR_offset) - 0.00145) < .0002)))
        if len(outliers) > 0:
            _ = plt.plot(p[outliers].scan_no, 1e3*p[outliers].LR_offset_wrap,
                         'ko', ms=8, mfc='none', mew=2, zorder=-100)
        _ = plt.title('R-L delay after subtracting mean value [%.0f MHz]' % (p.iloc[0].ref_freq))
        _ = plt.xlim(0, 1.05*plt.xlim()[1])
        wide(12, 5)
        plt.show()
        p.loc[outliers, "expt_no scan_id source timetag baseline ref_pol mbd_unwrap LR_offset LR_offset_wrap".split()]
    except:
        None